### Workflow:

- Before start:
    - Add files
    - Combine all files
    - import libraries
- Experiment:
    - EVERYTHING

# 1: Import libs and files

In [4]:
import pandas as pd


a = pd.read_csv('worldriskindex-datasets\worldriskindex-2000.csv')
a.head()

,Country,ISO3,W,E,V,S,C,A,S_01,S_02,...,AI_04a_Norm,AI_04a_Base,AI_04b_Norm,AI_04b_Base,AI_04c_Norm,AI_04c_Base,AI_05a_Norm,AI_05a_Base,AI_05b_Norm,AI_05b_Base
0,Afghanistan,AFG,4.19,0.25,70.32,61.91,78.59,71.47,65.90,70.66,...,100.00,24,91.54,24,92.31,7,73.35,147.35,57.41,6.18
1,Albania,ALB,6.53,2.38,17.94,12.58,44.15,10.40,53.08,15.47,...,19.61,97,19.81,97,25.19,92,48.84,1232.39,0.01,0.00
2,Algeria,DZA,10.29,2.65,39.94,30.07,51.99,40.76,48.02,35.44,...,40.23,86,42.01,86,43.75,78,44.09,1746.52,17.28,0.30
3,Andorra,AND,0.32,0.02,5.21,2.58,2.18,25.10,33.23,0.54,...,12.85,98,27.52,95,37.18,84,30.03,6003.08,50.93,4.07
4,Angola,AGO,11.06,2.18,56.09,35.52,65.02,76.41,59.61,64.24,...,84.23,31,100.00,21,80.24,26,56.41,756.01,93.27,325.00


In [ ]:
print("Help! Please\n :')")

Help! Please
 :')
